In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
speech = speech.reset_index()
# set s_id as index for final_df
speech.set_index('s_id', inplace=True)

In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
meal_data = meal_data.add_suffix('_meal')
play_data = play_data.add_suffix('_play')

In [ ]:
all_data = pd.concat([meal_data, play_data], axis=0)
all_data

In [ ]:
all_data['group'] = all_data['group'].map({'meal': 0, 'play': 1})


In [ ]:
X = all_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]
X

In [ ]:
y

# Getting scenario predictions

In [ ]:
all_data.dropna(inplace=True)
X = all_data[['Proportion speech child','Proportion speech caregiver', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
            'average_proximity','correlation', 'cross_correlation','avg interval duration' ,'avg silence duration',
       'variance_proximity', 'min_proximity','max_proximity'
 ]]


subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
            'average_proximity',
       'variance_proximity', 'min_proximity','max_proximity'
 ]

y = all_data['group']


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4,6],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
pipeline_gating = best_models['lr']['best_estimator']   

In [ ]:
pipeline_gating.predict_proba(X)

# Now get the the meal and play models predictions

In [ ]:

X = meal_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = meal_data['label'].iloc[:, 0]   



remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)
classifiers = [
('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
pipeline_play = best_models['svc_linear']['best_estimator']

In [ ]:
pipeline_meal = best_models['xgb']['best_estimator']

In [ ]:

X = play_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   



remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
X_meal = meal_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]
X_play = play_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]
y_meal = meal_data['label'].iloc[:,0]
y_play = play_data['label'].iloc[:,0]


In [ ]:
X_meal['group'] = X_meal['group'].map({'meal': 0})
X_play['group'] = X_play['group'].map({'play': 1})

In [ ]:
all_data[['Proportion speech child','Proportion speech caregiver', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
            'average_proximity','correlation', 'cross_correlation','avg interval duration' ,'avg silence duration',
       'variance_proximity', 'min_proximity','max_proximity'
 ]]
y_gating = all_data['group']

In [ ]:
combined_speeches = (pd.concat([X_meal['Proportion speech child'], X_play['Proportion speech child']], axis=1).dropna())
combined_speeches.columns = ['Proportion speech child_meal', 'Proportion speech child_play']    

In [ ]:
combined_movements = (pd.concat([X_meal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']], 
            X_play[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]], axis=1).dropna())
combined_movements.columns = ['cg_movement_mean_meal', 'cg_movement_var_meal', 'cg_movement_min_meal', 'cg_movement_max_meal',
            'child_movement_mean_meal', 'child_movement_var_meal', 'child_movement_min_meal', 'child_movement_max_meal',
            'cg_movement_mean_play', 'cg_movement_var_play', 'cg_movement_min_play', 'cg_movement_max_play',
            'child_movement_mean_play', 'child_movement_var_play', 'child_movement_min_play', 'child_movement_max_play']

In [ ]:
all_data['group']

# Stacking with gating

In [ ]:
from sklearn.model_selection import KFold, cross_val_predict
all_data = pd.concat([combined_speeches, combined_movements], axis=1)

subset_pca = ['cg_movement_mean_meal', 'cg_movement_var_meal', 'cg_movement_min_meal', 'cg_movement_max_meal',
            'child_movement_mean_meal', 'child_movement_var_meal', 'child_movement_min_meal', 'child_movement_max_meal',
            'cg_movement_mean_play', 'cg_movement_var_play', 'cg_movement_min_play', 'cg_movement_max_play',
            'child_movement_mean_play', 'child_movement_var_play', 'child_movement_min_play', 'child_movement_max_play']


remaining_features = [feat for feat in all_data.columns if feat not in subset_pca]
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])


meta_models = [
    ('svc', Pipeline([('preprocessor', preprocessor), ('svc', SVC(probability=True))])),
    ('logistic_regression', Pipeline([('preprocessor', preprocessor), ('logistic_regression', LogisticRegression())])),
    ('random_forest', Pipeline([('preprocessor', preprocessor), ('random_forest', RandomForestClassifier())]))
]


results = []
param_grids = {
    'svc': {
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear', 'rbf'],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4, 6, 8]
    },
    'logistic_regression': {
        'logistic_regression__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__solver': ['liblinear', 'saga'],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4, 6, 8]
    },
    'random_forest': {
        'random_forest__n_estimators': [10, 50, 100, 200],
        'random_forest__max_depth': [None, 5, 10, 15],
        'random_forest__min_samples_split': [2, 5, 10],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4, 6, 8]
    }
}

for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict_proba')
    play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict_proba')
    gating_predictions = cross_val_predict(pipeline_gating, X_gating, y_gating, cv=cv, method='predict_proba')
    gating_predictions_meal = gating_predictions[:len(X_meal)]
    gating_predictions_play = gating_predictions[len(X_meal):]
   # Concatenate predictions horizontally
    combined_predictions_meal = np.hstack((meal_predictions, gating_predictions_meal))
    combined_predictions_play = np.hstack((play_predictions, gating_predictions_play))
    
    # Create DataFrames for combined predictions
    combined_predictions_meal_df = pd.DataFrame(combined_predictions_meal, index=X_meal.index)
    combined_predictions_play_df = pd.DataFrame(combined_predictions_play, index=X_play.index)
    
    # Concatenate the combined predictions vertically
    combined_predictions_df = pd.concat([combined_predictions_meal_df, combined_predictions_play_df], axis=1)
    combined_predictions_df['Proportion speech child_meal'] = combined_speeches['Proportion speech child_meal']
    combined_predictions_df['Proportion speech child_play'] = combined_speeches['Proportion speech child_play']
    combined_predictions_df = pd.concat([combined_predictions_df, combined_movements], axis=1)
    y = pd.concat([y_meal, y_play], axis=1).dropna().iloc[:,0]
    combined_predictions_df.dropna(inplace=True)

    X = combined_predictions_df
    X.columns = X.columns.astype(str)

    for model_name, meta_model in meta_models:
        grid_search = GridSearchCV(meta_model, param_grids[model_name], cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X, y)
        best_meta_model = grid_search.best_estimator_
        
        accuracy_scores = cross_val_score(best_meta_model, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
        precision_scores = cross_val_score(best_meta_model, X, y,  cv=cv, scoring='precision_weighted', n_jobs=-1)
        recall_scores = cross_val_score(best_meta_model, X, y,  cv=cv, scoring='recall_weighted', n_jobs=-1)
        f1_scores = cross_val_score(best_meta_model, X, y, cv=cv, scoring='f1_weighted', n_jobs=-1)
        
        results.append({
            'random_state': i,
            'model': model_name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'f1_score': f1_scores.mean()
        })
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('model').agg(['mean', 'std'])

In [ ]:
X_gating = all_data[['Proportion speech child','Proportion speech caregiver', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
            'average_proximity','correlation', 'cross_correlation','avg interval duration' ,'avg silence duration',
       'variance_proximity', 'min_proximity','max_proximity'
 ]]
y_gating = all_data['group']

In [ ]:
gating_predictions

In [ ]:
all_data.columns

# use the predictions probabilities as weights for the features

In [ ]:
pd.concat([X, all_data['group']], axis=1)

In [ ]:
from sklearn.model_selection import KFold, cross_val_predict

data = all_data[['Proportion speech child', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max','group']]
subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
         ]


remaining_features = [feat for feat in data.columns if feat not in subset_pca]
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

meta_models = [
    ('svc', Pipeline([('preprocessor', preprocessor), ('svc', SVC(probability=True))])),
    ('logistic_regression', Pipeline([('preprocessor', preprocessor), ('logistic_regression', LogisticRegression())])),
    ('random_forest', Pipeline([('preprocessor', preprocessor), ('random_forest', RandomForestClassifier())])),
    ('xgb', Pipeline([('preprocessor', preprocessor), ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))]))
]


results = []
param_grids = {
    'svc': {
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear', 'rbf'],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    },
    'logistic_regression': {
        'logistic_regression__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__solver': ['liblinear', 'saga'],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    },
    'random_forest': {
        'random_forest__n_estimators': [10, 50, 100, 200],
        'random_forest__max_depth': [None, 5, 10, 15],
        'random_forest__min_samples_split': [2, 5, 10],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    },
    'xgb': {
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    }
}

for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict_proba')
    play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict_proba')
    gating_predictions = cross_val_predict(pipeline_gating, X_gating, y_gating, cv=cv, method='predict_proba')
    weights_meal = meal_predictions[:, 1]
    weights_play = play_predictions[:, 1]
    
    # Use gated predictions as weights for raw features
    weighted_X_meal = X_meal.multiply(weights_meal, axis=0)
    weighted_X_play = X_play.multiply(weights_play, axis=0)
   # Concatenate predictions horizontally
    X = pd.concat([weighted_X_meal, weighted_X_play], axis=0)
    X = pd.concat([X, all_data['group']], axis=1)

    y = pd.concat([y_meal, y_play], axis=0)
    combined_predictions_df.dropna(inplace=True)
    groups = X.index
    cv = GroupKFold(n_splits=5)

    for model_name, meta_model in meta_models:
        grid_search = GridSearchCV(meta_model, param_grids[model_name], cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X, y, groups=groups)
        best_meta_model = grid_search.best_estimator_
        
        accuracy_scores = cross_val_score(best_meta_model, X, y, groups=groups,cv=cv, scoring='accuracy', n_jobs=-1)
        precision_scores = cross_val_score(best_meta_model, X, y, groups=groups, cv=cv, scoring='precision_weighted', n_jobs=-1)
        recall_scores = cross_val_score(best_meta_model, X, y,  groups=groups,cv=cv, scoring='recall_weighted', n_jobs=-1)
        f1_scores = cross_val_score(best_meta_model, X, y, groups=groups,cv=cv, scoring='f1_weighted', n_jobs=-1)
        
        results.append({
            'random_state': i,
            'model': model_name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'f1_score': f1_scores.mean()
        })
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('model').agg(['mean', 'std'])

In [ ]:
y_label_meals = pd.concat([y_meal, y_play], axis=0)
y_label_plays = pd.concat([y_meal, y_play], axis=0)


In [ ]:
gating_predictions[:,1]

# Multi task learning

In [ ]:
# Define the data and subset for PCA
from sklearn.multioutput import MultiOutputClassifier
data = pd.concat([X_meal, X_play], axis=0)
subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
              'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

remaining_features = [feat for feat in data.columns if feat not in subset_pca]
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the base models
xgb_model = MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
rf_model = MultiOutputClassifier(RandomForestClassifier())

# Define the parameter grids
param_grids = {
    'xgb': {
        'xgb__estimator__n_estimators': [50, 100, 200],
        'xgb__estimator__learning_rate': [0.01, 0.1, 0.2],
        'xgb__estimator__max_depth': [3, 5, 10],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    },
    'rf': {
        'rf__estimator__n_estimators': [10, 50, 100, 200],
        'rf__estimator__max_depth': [None, 5, 10, 15],
        'rf__estimator__min_samples_split': [2, 5, 10],
        'preprocessor__pca_pipeline__pca__n_components': [2, 4]
    }
}
results = []
for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    y_combined = pd.concat([y_label_meals, y_label_plays], axis=1)
    gating_predictions = cross_val_predict(pipeline_gating, X_gating, y_gating, cv=cv, method='predict_proba')
    groups = data.index
    data['gating_0'] = gating_predictions[:,0]
    data['gating_1'] = gating_predictions[:,1]
    X_shuffled, y_shuffled, groups = shuffle(data, y_combined, groups, random_state=i)
    # Combine the labels for multi-task learning

    # Shuffle the data

    # Define the cross-validation strategy
    cv = GroupKFold(n_splits=5)

    # Initialize a list to store the results


    # Evaluate XGBoost
    xgb_pipeline = Pipeline([('preprocessor', preprocessor), ('xgb', xgb_model)])
    xgb_grid_search = GridSearchCV(xgb_pipeline, param_grids['xgb'], cv=cv, scoring='accuracy', n_jobs=-1)
    xgb_grid_search.fit(X_shuffled, y_shuffled, groups = groups)

    # Calculate the cross-validated scores
    xgb_accuracy_scores = cross_val_score(xgb_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='accuracy', n_jobs=-1)
    xgb_f1_scores = cross_val_score(xgb_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='f1_weighted', n_jobs=-1)
    xgb_precision_scores = cross_val_score(xgb_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='precision_weighted', n_jobs=-1)
    xgb_recall_scores = cross_val_score(xgb_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='recall_weighted', n_jobs=-1)

    # Store the results
    results.append({
        'model': 'xgb',
        'best_params': xgb_grid_search.best_params_,
        'accuracy': xgb_accuracy_scores.mean(),
        'f1_score': xgb_f1_scores.mean(),
        'precision': xgb_precision_scores.mean(),
        'recall': xgb_recall_scores.mean()
    })

    # Evaluate Random Forest
    rf_pipeline = Pipeline([('preprocessor', preprocessor), ('rf', rf_model)])
    rf_grid_search = GridSearchCV(rf_pipeline, param_grids['rf'], cv=cv, scoring='accuracy', n_jobs=-1)
    rf_grid_search.fit(X_shuffled, y_shuffled, groups=groups)

    # Calculate the cross-validated scores
    rf_accuracy_scores = cross_val_score(rf_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='accuracy', n_jobs=-1)
    rf_f1_scores = cross_val_score(rf_grid_search.best_estimator_, X_shuffled, y_shuffled, groups = groups,cv=cv, scoring='f1_weighted', n_jobs=-1)
    rf_precision_scores = cross_val_score(rf_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='precision_weighted', n_jobs=-1)
    rf_recall_scores = cross_val_score(rf_grid_search.best_estimator_, X_shuffled, y_shuffled,groups = groups, cv=cv, scoring='recall_weighted', n_jobs=-1)

    # Store the results
    results.append({
        'model': 'rf',
        'best_params': rf_grid_search.best_params_,
        'accuracy': rf_accuracy_scores.mean(),
        'f1_score': rf_f1_scores.mean(),
        'precision': rf_precision_scores.mean(),
        'recall': rf_recall_scores.mean()
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Print the results
print(results_df)

In [ ]:
results_df.groupby('model').mean()